In [1]:
import numpy as np
import matplotlib.pylab as plt
import glob
import time
import imageio
import io
import cv2
import time
import scipy.spatial
import imageio
import io
import cv2
import random
import matplotlib.gridspec as gridspec
import time

## Load adversarial attacks and computer: Delta, Fourier Spectrum, L2 Distance and Maximum Amplitude of Fourier Spectrum

In [2]:
def rgb2gray(rgb):

    r, g, b = rgb[0,:,:], rgb[1,:,:], rgb[2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def get_fourier(imag):
    image_sample = np.fft.fft2(imag)
    F_orig = np.fft.fftshift(image_sample)
    F_orig = np.log(np.abs(F_orig))
    return F_orig

model = 'lck15'

image_total = []
delta_total = []
distance_total = []
fourier_total = []
maximum_total = []
original_total = []
adversarial_total = []
normal_total = []
m = time.time()
namess = './cifar_resnet/ckpt_{}_*_*_linf_adversarial_100.npy'.format(model)
namess = glob.glob(namess)
maxss = []
for name in namess:
    name = name.split('/')[-1].split('_')[2]
    maxss.append(int(name))
maxs = max(maxss)
mins = min(maxss)
for i in range(mins,maxs+1):
    for h in range(1,392,30):
        name = './cifar_resnet/ckpt_{}_{}_{}_linf_adversarial_100.npy'.format(model,i,h)
        b = np.load(name, allow_pickle=True)
        b = b[0,:,:]
        images = []
        deltas = []
        distances = []
        fouriers = []
        maximums = []
        originals = []
        adversarials = []
        normals = []
        for j in range(b.shape[0]):
            if b[j,3] != None:
                normals.append(b[j,1])
                delta = b[j,0] - b[j,1]
                delta = rgb2gray(delta)
                deltas.append(delta)
                fourier = get_fourier(delta)
                fouriers.append(fourier)
                maximums.append(fourier.max())
                original = b[j,1].flatten()
                adversarial = b[j,0].flatten()
                distance = scipy.spatial.distance.euclidean(original, adversarial)
                images.append(b[j,0])
                distances.append(distance)
                originals.append(b[j,2])
                adversarials.append(b[j,3])
            else:
                normals.append(np.zeros((3,32,32)))
                delta = np.zeros((32,32))
                deltas.append(delta)
                images.append(np.zeros((3,32,32)))
                distances.append(None)
                fouriers.append(delta)
                maximums.append(None)
                originals.append(b[j,2])
                adversarials.append(None)

        
        images = np.array(images)
        deltas = np.array(deltas)
        normals = np.array(normals)
        distances = np.array(distances)
        fouriers = np.array(fouriers)
        maximums = np.array(maximums)
        originals = np.array(originals)
        adversarials = np.array(adversarials)
        #images = images.mean()
        image_total.append(images)
        delta_total.append(deltas)
        distance_total.append(distances)
        fourier_total.append(fouriers)
        maximum_total.append(maximums)
        original_total.append(originals)
        adversarial_total.append(adversarials)
        normal_total.append(normals)
        if (i*391 + j) %25 ==0:
            print(i, j, time.time() - m)  
            m = time.time()
    
image_total = np.array(image_total)
delta_total = np.array(delta_total)
distance_total = np.array(distance_total)
fourier_total = np.array(fourier_total)
maximum_total = np.array(maximum_total)
original_total = np.array(original_total)
normal_total = np.array(normal_total)
adversarial_total = np.array(adversarial_total)

np.save('./cifar_resnet/cifar_dynamics_adv_normal_{}.npy'.format(model), normal_total)
np.save('./cifar_resnet/cifar_dynamics_adv_image_{}.npy'.format(model), image_total)
np.save('./cifar_resnet/cifar_dynamics_adv_delta_{}.npy'.format(model), delta_total)
np.save('./cifar_resnet/cifar_dynamics_adv_distance_{}.npy'.format(model), distance_total)
np.save('./cifar_resnet/cifar_dynamics_adv_fourier_{}.npy'.format(model), fourier_total)
np.save('./cifar_resnet/cifar_dynamics_adv_maximum_{}.npy'.format(model), maximum_total)
np.save('./cifar_resnet/cifar_dynamics_adv_original_{}.npy'.format(model), original_total)
np.save('./cifar_resnet/cifar_dynamics_adv_adversarial_{}.npy'.format(model), adversarial_total)

## Use if you want to load the already saved files

## Main Visualization: 6 columns with 8 examples (one in each row). Visualizes [Original Image, Adversarial Image, Delta (normalize within a frame), Fourier Spectrum of the Delta. Maximum Amplitude through time, L2 Distance through Time]

In [ ]:
%%capture

def get_img_from_fig(fig, dpi=100):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img


def dyn_fourier(normal, image, delta, distance, fourier, maximum, original, adversarial, dist_max, max_max, idx, i):
    fig = plt.figure(figsize=(16, 16))
    #fig, axs = plt.subplots(8,5, figsize=(40, 20), facecolor='w', edgecolor='k')
    gs1 = gridspec.GridSpec(nrows=8, ncols=6, left=0.05, right=1.0,
    wspace=0.5, hspace=0.5)
    
    for j in range(8):
        axs = fig.add_subplot(gs1[j, 0])
        axs.imshow(np.moveaxis(normal[idx[j],...], 0, -1), vmin=0.0, vmax=1.0, interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 13), int(i % 13) * 30 +1,original[idx[j]], adversarial[idx[j]]))
        
        axs = fig.add_subplot(gs1[j, 1])
        axs.imshow(np.moveaxis(image[idx[j],...], 0, -1), vmin=0.0, vmax=1.0, interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 13), int(i % 13) * 30 + 1,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 2])
        axs.imshow(delta[idx[j],...], interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 13), int(i % 13)* 30 + 1,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 3])
        axs.imshow(fourier[idx[j],...],vmin=0.0, interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 13), int(i % 13)* 30 + 1,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 4])
        axs.plot(maximum[:,idx[j]])
        axs.set_ylim(0,max_max)
        axs.set_ylabel('Maximum')
        axs.set_title('E:{} I:{}, C:{}. A:{}'.format(int(i // 13), int(i % 13)* 30 + 1,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 5])
        axs.plot(distance[:,idx[j]])
        axs.set_ylim(0,dist_max)
        axs.set_ylabel('L2 Norm')
        axs.set_title('E:{} I:{}, C: {}. A:{}'.format(int(i // 13), int(i % 13)* 30 + 1,original[idx[j]], adversarial[idx[j]]))
    image = get_img_from_fig(fig)
    del fig
    return image

vmax = image_total.max()
kargs = { 'macro_block_size': None }
idx = [i for i in range(image_total[0].shape[0])]
imageio.mimsave('./cifar_resnet/image_total_dynamics_cifar_{}.mp4'.format(model), [dyn_fourier(normal_total[i,...], image_total[i,...], delta_total[i,...], 
                                                               distance_total, fourier_total[i,...], 
                                                               maximum_total, original_total[i,...], adversarial_total[i,...], distance_total[distance_total !=None].max(), maximum_total[maximum_total !=None].max(),idx,i) 
                                                   for i in range(image_total.shape[0])], fps=60, **kargs)